# 8.28
## T1. [Five Dimensional Points](https://codeforces.com/problemset/problem/850/A)
- **题意**: 给定一个5维空间中的n个点，计算其中“完美点”的个数
- **题解**: 考虑**上限情况**，然后暴力求解，事实上，需要考虑的点个数最多不会超过$2\times k + 1$个点
___
**Proposition**: 给定$k$维空间的$n$个点，构造两两夹角非锐角的向量，最多需要考虑的节点个数不超过$2\times k + 1$

**Proof**: 在$k$维空间中，固定一个点，构造两两间夹角非锐角的向量，个数最多的情况显然是两两正交，此时向量个数最多为$k$(线性空间的维度)，考虑正负两个方向，此时需要计算的点个数为$2\times k$，再考虑出发点，最终需要考虑的点个数最多为$2 \times k + 1$，当节点个数超过此值时，必然会出现两个向量夹角为锐角.

In [ ]:
def vec(a, b):
    return [y - x for x, y in zip(a, b)]

def cal(a, b): 
    return sum([x * y for x, y in zip(a, b)])

n = int(input()) 
if n >= 12: 
    print(0)
    exit()
pts = [list(map(int, input().split())) for _ in range(n)]

outs = [] 

for i in range(n): 
    ok = True 
    for j in range(n): 
        for k in range(j):
            if cal(vec(pts[i], pts[j]), vec(pts[i], pts[k])) > 0: 
                ok = False 
    if ok: 
        outs.append(f"{i + 1}")

print(len(outs))
print(" ".join(outs))

## T2. [Robo-Footballer](https://codeforces.com/problemset/problem/248/C)
- **题意**: 在y轴上有一个区间，表示一个球门，给定球的位置$(x_b, y_b)$，需要通过将球踢到墙($y=y_w$)上，通过折射之后让球通过球门
- **题解**:
  本题主要考察问题转换:  
  1. 考虑球门的镜像区间, 将折射问题简化为求直线交点
  2. 考虑到球的直径$r$, 判断条件转换为判断镜像后区间的下端点与球的线路的最短距离是否超过$r$

In [ ]:
y1, y2, yw, xb, yb, r = map(int, input().split())
y1, y2 = 2 * yw - 2 * r - y2, 2 * yw - 2 * r - y1 

if r ** 2 * ((yb - y2 + r) ** 2 + xb ** 2) > xb ** 2 * (y2 - y1 - r) ** 2: 
    print(-1)
else:
    print((yw - y2) * xb / (yb - y2 + r))